# Model updated with the matlab implementation written by the authors 

In [1]:
#import sys
#sys.setrecursionlimit(100000)

# computer algebra library
import symengine
import sympy
sympy.init_printing()

# jupyter notebook display library
from IPython.display import display as IPyDisplay
from IPython.display import Latex

# numeric integration library
import scipy
from scipy.integrate import solve_ivp

# plotting library

from matplotlib.pyplot import plot,show

# library for scientific computing, used here for plotting
import numpy

# import dirac delta function
from sympy import DiracDelta

# hyperbolic functions
# from mpmath import tanh, mp
from sympy.functions.elementary.hyperbolic import tanh

# Some auxiliary Python functions

In [2]:
PyPrint = print

def doNothing(*ka, **kwa): pass

## Decomment the following lines if you want to disable printing:
## useful if the rendering is very slow.
# print = doNothing
# display = doNothing

## To restore the original printing functions, do this:
# print = PyPrint
# display = IPyDisplay

# Custom printing functions

def latexColor(s, color):
    return "{\\color{%s}{" % color + s + '}}'

def vPhantom(obj):
    objt = type(obj)
    if objt in (list, tuple, set, dict):
        res = ''
        objlist = list(obj.keys()) if objt == dict else tuple(obj)
        for o in objlist:
            res += " " + vPhantom(o)
            if objt == dict:
                res += " " + vPhantom(obj[o])
    else:
        res = "\\vphantom{%s}" % sympy.latex(obj)
    return res
                
def toCustomLatex(
    obj, 
    recursive=False, 
    delimiters=True,
    separator=',',
    spacer = '\\quad',
    color='blue'
):
    objt = type(obj)

    Color = lambda x: latexColor(x, color)
    text2latex = lambda x: Color("\\text{%s}" % str(x))
    
    if type(obj) == str:
        return text2latex(obj)
    
    objlist = list(obj.keys()) if objt == dict else tuple(obj)
    latex_obj = ''
    
    toRecLatex = lambda x: toCustomLatex(
        x, 
        recursive = recursive,
        delimiters = True,
        color = color
    )
    
    for ii in range(len(objlist)):
        
        if type(objlist[ii]) in (set, dict, list, tuple) and recursive:
            toLatex = toRecLatex
        elif type(objlist[ii]) == str:
            toLatex = text2latex
        else:
            toLatex = lambda x: Color(sympy.latex(x))
        
        try:
            o = toLatex(objlist[ii])
        except:
            o = text2latex(o)

        if objt == dict:
            try:
                o += ": " + toLatex(obj[objlist[ii]])
            except:
                o += ": " + text2latex(obj[objlist[ii]])
            
        sep = (ii + 1 < len(objlist)) * (Color(separator) + spacer + '\n')
        latex_obj += o + sep

    if delimiters:
        ldelims, rdelims = {
            set: "\{ \}",
            dict: "\{ \}",
            list: "[ ]",
            tuple: "( )"
        }[objt].split(" ")

        ldelim = \
            "\\left " + ldelims + vPhantom(obj) + "\\right .\n"
        rdelim = \
            "\\left . " + vPhantom(obj) + "\\right " + rdelims + "\n"

        latex_obj = Color(ldelim) + latex_obj + Color(rdelim)

    return latex_obj

def customDisplay(
    obj,
    recursive=False,
    color='blue',
    textcolor='brown',
    **args
):
    objt = type(obj)
    
    Color = lambda x: latexColor(x, color)
    
    if objt in (set, dict, list, tuple):
        try:
            latex_obj = toCustomLatex(
                obj,
                recursive = recursive,
                color = color,
                **args
            )
        except:
            latex_obj = toCustomLatex(str(obj))
        
        latex_obj =   "\\begin{align*}\n\\begin{autobreak}\n" \
                    + latex_obj \
                    + "\\end{autobreak}\n\\end{align*}\n" \
        
        display(Latex(latex_obj))
        return

    
    if objt == str:
        display(Latex("$%s$" % latexColor("\\text{%s}" % obj, textcolor)))
        return
    
    try:
        obj_latex = sympy.latex(obj)
        display(Latex("$$%s$$" % Color(obj_latex)))
    except:
        try:
            customDisplay(str(obj))
        except:
            display(obj)
            
def blockDisplay(*args, **kwargs):
    for a in args:
        customDisplay(a, **kwargs)
        
# Precision used for the definition of numeric values in Sympy 
# equations.
precision = 50

def NVal(val):
    """
        Shortcut to create Sympy values with arbitrary precision.
    """
    return sympy.N(val, precision)

def chop(obj, low_precision=2):
    """
        Truncate the very long floating point values of the Sympy
        object obj. This function is meant to be used only for 
        displaying, to make Sympy objects readable.
    """
    if isinstance(obj, dict):
        return {kk: chop(obj[kk]) for kk in obj.keys()}
    for typ in [list, tuple, set]:
        if isinstance(obj, typ):
            return typ(chop(o) for o in obj)
    if type(obj) in (sympy.Float, float):
        exponent = -sympy.floor(sympy.log(sympy.Abs(obj))\
                                / sympy.log(10.)) + low_precision
        return sympy.N(sympy.Integer(obj * 10**exponent)\
                       / 10**exponent, precision)
    sfloat = sympy.Wild(
        'sfloat', 
        properties = [lambda x: isinstance(x, sympy.Float)]
    )
    if str(type(obj))[8:13] == 'sympy':
        return sympy.N(obj.replace(
                            sfloat, 
                            lambda sfloat: chop(sfloat))
                      )
    return obj

#### Substitution functions

In [3]:
def sortSubsDict(subs_dict):
    """
        Given a dictionary of unsorted substitution dictionaries, 
        return a sorted list of substitution dictionaries without
        conflicts. 
        
        A conflict happens when one of the terms to be replaced is
        contained in another term to be replaced. 
        For example, the expression 1 + G(t + 2) contains the
        expression t + 2. If both must be replaced at the same
        time, there is a conflict, since replacing t + 2 may
        make impossible to replace 1 + G(t + 2).
        
        In this case, the substitutions involving "bigger terms"
        must be applied first.
        
        The returned list of dictionaries is built to guarantee
        that there is no conflict between substitutions of the
        same dictionary, and the expressions involved in the
        substitutions of each dictionary are not contained
        in the subsequent dictionaries of the returned list.
        
        Accepted types: dictionary
    """
    dict_list = []
    keys = list(subs_dict.keys())
    while keys:
        dict_list += [dict()]
        for s in keys:
            if sum(len(x.find(s)) for x in keys) == 1:
                dict_list[-1].update({s: subs_dict[s]})
        for s in dict_list[-1].keys():
            keys.remove(s)
    return dict_list

def subs_aux(
    obj, 
    subs_dict_or_list,
    RHSonly=False,
    simplify=True,
    doit=True,
    recursive=False,
    maxRecursionDepth=10
):
    """
        Apply the given list of substitutions subs_dict_or_list
        to the object obj.
        
        Accepted types for the object:
            sympy equations or expressions or anything 
                having the .subs() method,
            dict, list, tuple, set
            
        Accepted types for the substitutions:
            list of pairs,
            dictionary of substitutions

        An object of the same type of obj is returned.
        If the object is of type dictionary, the substitutions
        are NOT applied to its keys.
        
        If RHSonly is True and the object is of type sympy.Eq,
        the substitutions are applied only to the right hand
        side of the equation.
        In any case, simplify() and doit() are applied separately
        to each side of an equation.
        
        If an equation becomes trivially True, it is removed from
        the dictionary or list or set or tuple.
    """
    
    if type(subs_dict_or_list) == dict:
        subs_list = [(kk, subs_dict_or_list[kk]) 
                     for kk in subs_dict_or_list.keys()]
    else:
        subs_list = list(subs_dict_or_list)
    
    subss_aux = lambda x: subs_aux(
        x, 
        subs_list, 
        RHSonly = RHSonly, 
        simplify = simplify,
        doit = doit,
        recursive = recursive
    )
    
    notTrivial = \
        lambda x: not isinstance(x, sympy.boolalg.BooleanTrue)
        
    objt = type(obj)
    if objt == dict:
        res = dict()
        for kk in obj.keys():
            objsub = subss_aux(obj[kk])
            if notTrivial(objsub):
                res.update({kk: objsub})
    elif objt in (list, tuple, set):
        res = objt()
        for kk in obj:
            skk = subss_aux(kk)
            if notTrivial(skk):
                if objt == list:
                    res.append(skk)
                elif objt == tuple:
                    res += (skk,)
                else:
                    res.add(skk)
    else:
        if isinstance(objt, sympy.Eq):
            if RHSonly:
                lhs = obj.args[0]
            else:
                lhs = subss_aux(obj.args[0])
            rhs = subss_aux(obj.args[1])
            res = sympy.Eq(lhs, rhs)
        else:
            res = obj.subs(subs_list, simplify=False)
            if recursive:
                oldres = obj
                for ii in range(maxRecursionDepth):
                    if res != oldres:
                        oldres = res
                        res = res.subs(subs_list, simplify=False)
                    else:
                        break
            if simplify:
                res = res.simplify()
            if doit:
                res = res.doit()
    return res

def subs(
    obj, 
    subs_dict,
    RHSonly=False,
    simplify=True,
    doit=True,
    recursive=False,
    maxRecursionDepth=10
):
    res = obj
    subs_dict_list = sortSubsDict(subs_dict)
    for sd in subs_dict_list:
        res = subs_aux(
            res,
            sd, 
            RHSonly = RHSonly, 
            simplify = simplify, 
            doit = doit,
            recursive = recursive,
            maxRecursionDepth = maxRecursionDepth
        )
    return res

# Model

In [4]:
class EmptyObject(): pass

model = EmptyObject()

symbols = EmptyObject()
model.symbols = symbols

def SSymbol(s, **kwargs):
    return sympy.Symbol(s, nonnegative=True, **kwargs)

def SFunction(sf, **kwargs):
    return sympy.Function(sf, nonnegative=True, **kwargs)

## Variables, Parameters and constants

In [5]:
# Variables
t      = sympy.Symbol('t')

Gp     = SFunction('G_{\\textit{p}}')(t) # glucose masses in plasma and rapidly equilibrating tissues (mg/kg)
Gt     = SFunction('G_{\\textit{t}}')(t) # glucose masses in plasma and slowly equilibrating tissues (mg/kg)
G      = SFunction('G')(t)   # plasma glucose concentration (mg/dl)
EGP    = SFunction('EGP')(t) # endogenous glucose production (mg/kg/min)
Ra     = SFunction('R_{\\textit{a}}')(t)  # appearance rate of glucose in plasma (mg/kg/min)
E      = SFunction('E')(t)   # renal excretion (mg/kg/min)
Uii    = SFunction('U_{\\textit{ii}}')(t) # insulin-independent glucose utilizations (mg/kg/min)
Uid    = SFunction('U_{\\textit{id}}')(t) # insulin-dependent glucose utilizations (mg/kg/min)
U      = SFunction('U')(t)   # total glucose utilizations (mg/kg/min)

Ip     = SFunction('I_{\\textit{p}}')(t) # Insulin masses in plasma (pmol/kg)
Il     = SFunction('I_{\\textit{l}}')(t) # Insulin masses in liver (pmol/kg)
I      = SFunction('I')(t)   # Insulin concentration (pmol/l)
S      = SFunction('S')(t)   # Insulin secretion (pmol/kg/min)
m3     = SFunction('m_3')(t) # ??
HE     = SFunction('HE')(t)  # Hepatic Extraction of insulin
Ipo    = SFunction('I_{\\textit{po}}')(t) # Amount of insulin in the portal vein (pmol/kg)
Id     = SFunction('I_{\\textit{d}}')(t)  # delayed insulin signal realized with 2 compartments (pmol/l)
I1     = SFunction('I_1')(t) # used by Id (for delayed insulin signal)

Qsto  = SFunction('Q_{\\textit{sto}}')(t)   #  amount of glucose in the stomach (mg) = Qsto1 + Qsto2
Qsto1 = SFunction('Q_{\\textit{sto1}}')(t)  #  amount of solid glucose in the stomach (mg)
Qsto2 = SFunction('Q_{\\textit{sto2}}')(t)  #  amount of liquid glucose in the stomach (mg)
Qgut  = SFunction('Q_{\\textit{gut}}')(t)   #  glucose mass in the intestin (mg)
kempt = SFunction('K_{\\textit{empt}}')(t)  #  rate "constant" of gastric emptying (1/min)

X     = SFunction('X')(t)                   # insulin in the interstitial fluid (pmol/L)
Y     = SFunction('Y')(t)                   # not defined ...
Spo   = SFunction('S_{\\textit{po}}')(t)    # not defined ...

symbols.time = {t}
symbols.variables = [Gp, Gt, G, EGP, Ra, E, Uii, Uid, U, Ip, Il, I, S, m3, HE, Ipo, Id, I1, Qsto, Qsto1, 
                     Qsto2, Qgut, kempt, X, Y, Spo]

# Parameters
k1    = SSymbol('k_1')      # rate parameter (1/min) for glucose model
k2    = SSymbol('k_2')      # rate parameter (1/min) for glucose model
VG    = SSymbol('V_G')      # distribution volume of glucose (dl/kg)
VI    = SSymbol('V_I')      # distribution volume of insulin (l/kg)
m1    = SSymbol('m_1')      # rate parameter (1/min) for insulin model
m2    = SSymbol('m_2')      # rate parameter (1/min) for insulin model
m4    = SSymbol('m_4')      # rate parameter (1/min) for insulin model
m5    = SSymbol('m_5')      # rate parameter (1/min) for insulin model
m6    = SSymbol('m_6')      # rate parameter (1/min) for insulin model
kmin  = SSymbol('k_{min}') # min value of kempt (1/min)
kmax  = SSymbol('k_{max}') # max value of kempt (1/min)
kabs  = SSymbol('k_{abs}')  # rate constant of intestinale absorption (1/min)
kgri  = SSymbol('k_{gri}')  # rate of grinding (1/min)
f     = SSymbol('f')        # fraction of intestinal absorption which actually appears in plasma
b     = SSymbol('b')        # percentage of the dose for which kempt decreases at (kmax-kmin)/2
c     = SSymbol('c')        # percentage of the dose for which kempt is back to (kmax-kmin)/2
kp1   = SSymbol('k_{p1}')   # extrapolated at zero glucose and insulin (mg/kg/min)
kp2   = SSymbol('k_{p2}')   # liver glucose effectiveness (1/min)
kp3   = SSymbol('k_{p3}')   # parameter governing amplitude of insulin action on the liver (mg/kg/min per pmol/l)
kp4   = SSymbol('k_{p4}')   # parameter governing amplitude of portal insulin action on the liver (mg/kg/\min /(pmol/kg))
ki    = SSymbol('k_i')      # rate parameter accounting for delay between insulin signal and insulin action (1/min)
Fcns  = SSymbol('F_{cns}')  # rate of glucose uptake by the brain and erythrocytes (mg/kg/min)
p2U   = SSymbol('p_{\\textit{2U}}') # rate constant of insulin action on the peripheral glucose utilization (1/min)
Vm0   = SSymbol('V_{\\textit{m0}}') # (mg/kg/min)
Vmx   = SSymbol('V_{\\textit{mx}}') # (mg/kg/min per pmol/l)
Km0   = SSymbol('K_{\\textit{m0}}') # (mg/kg)
Kmx   = SSymbol('K_{\\textit{mx}}') # (mg/kg)
γ     = SSymbol('\\gamma')  # transfer rate constant between portal vein and liver (1/min)
K     = SSymbol('K')        # pancreatic responsivity to the glucose rate of change (pmol/kg per mg/dl)
α     = SSymbol('\\alpha')  # delay between glucose signal and insulin secretion (1/min)
β     = SSymbol('\\beta')   # pancreatic responsivity to glucose (pmol/kg/min per mg/dl)
h     = SSymbol('h')        # threshold of glucose above which the cells initiate to produce new insulin (mg/dl)
ke1   = SSymbol('k_{\\textit{e}_1}') # glomerular filtration rate (1/min)
ke2   = SSymbol('k_{\\textit{e}_2}') # renal threshold of glucose (mg/kg)

# memo: h has been set equal to Gb

symbols.parameters = [ k1, k2, VG, VI, m1, m2, m4, m5, m6, kmin, kmax, kabs, kgri, f, b, c, kp1, kp2, kp3, kp4, 
                      ki, Fcns, p2U, Vm0, Vmx, Km0, Kmx, γ, K, α, β, h, ke1, ke2 ]

# Variables at basal steady state
Gpb  = SSymbol('G_{\\text{pb}}')  # Gp at basal state
Gtb  = SSymbol('G_{\\text{tb}}')  # Gt at basal state
Gb   = SSymbol('G_{\\text{b}}')   # G at basal state
EGPb = SSymbol('EGP_{\\text{b}}') # EGP at basal state
Rab  = SSymbol('R_{\\textit{ab}}')
Eb   = SSymbol('E_{\\text{b}}')   # E at basal state
Uiib = SSymbol('U_{\\text{iib}}') # Uii at basal state
Uidb = SSymbol('U_{\\text{idb}}') # Uid at basal state
Ub   = SSymbol('U_{\\text{b}}')   # Ub at basal state (ie. Uiib+Uid)

Ipb    = SSymbol('I_{\\text{pb}}')   # Ip at basal state
Ilb    = SSymbol('I_{\\text{lb}}')   # Il at basal state
Ib     = SSymbol('I_{\\text{b}}')    # I at basal state
Sb     = SSymbol('S_{\\text{b}}')    # S at basal state
m3b    = SSymbol('m_{\\text{3b}}')   # m3 at basal state
HEb    = SSymbol('HE_{\\text{b}}')   # HE at basal state
Ipob   = SSymbol('I_{\\text{pob}}')  # Ipo at basal state
Idb    = SSymbol('I_{\\text{db}}')   # Id at basal state
I1b    = SSymbol('I_{\\text{1b}}')   # I1 at basal state

Qstob  = SSymbol('Q_{\\textit{stob}}')
Qsto1b = SSymbol('Q_{\\textit{sto1b}}')
Qsto2b = SSymbol('Q_{\\textit{sto2b}}')
Qgutb  = SSymbol('Q_{\\textit{gutb}}')
kemptb = SSymbol('K_{\\textit{emptb}}')

Xb     = SSymbol('X_{\\textit{b}}')
Yb     = SSymbol('Y_{\\textit{b}}')
Spob   = SSymbol('S_{\\textit{pob}}')

symbols.variables_basal = [ Gpb, Gtb, Gb, EGPb, Rab, Eb, Uiib, Uidb, Ub, Ipb, Ilb, Ib, Sb, m3b, HEb,
                            Ipob, Idb, I1b, Qstob, Qsto1b, Qsto2b, Qgutb, kemptb, Xb, Yb, Spob ]

# input variables
D     = SFunction('D')(t) # we set D a function with null derivative instead of a constant parameters to make easier the the updating of its value
# D     = SSymbol('D')        # amount of ingested glucose (mg)
BW    = SSymbol('BW')       # body weight (kg)
symbols.input_variables = { Gb, D, BW }

## Equations

In [6]:
equations = dict()
model.equations = equations

### Glucose subsystem

Equations (1)

_We add Max function on EGP and Ra as in the matlab implementation_

In [7]:
equations.update({
    Gp : sympy.Eq(Gp.diff(), sympy.Max(EGP,0) + sympy.Max(Ra,0) - Uii - E - k1*Gp + k2*Gt),
    #Gp : sympy.Eq(Gp.diff(), EGP + Ra - Uii - E - k1*Gp + k2*Gt),
    Gt : sympy.Eq(Gt.diff(), -Uid + k1*Gp - k2*Gt),
    G  : sympy.Eq(G, Gp / VG),
})
blockDisplay(*(equations[e] for e in [Gp, Gt, G]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

##### Endogenous Glucose Production 

Equations (10)

In [8]:
equations.update({
    EGP  : sympy.Eq(EGP, kp1 - kp2*Gp - kp3*Id - kp4*Ipo),
})
blockDisplay(*(equations[e] for e in [EGP]))

<IPython.core.display.Latex object>

##### Glucose rate of appearance

Equations (13)

In [9]:
equations.update({
    Qsto  : sympy.Eq(Qsto, Qsto1 + Qsto2),
    Qsto1 : sympy.Eq(Qsto1.diff(), -kgri * Qsto1 + D*DiracDelta(t)),
    Qsto2 : sympy.Eq(Qsto2.diff(), -kempt*Qsto2 + kgri*Qsto1),
    Qgut  : sympy.Eq(Qgut.diff(), -kabs*Qgut + kempt*Qsto2),
    Ra    : sympy.Eq(Ra, f*kabs*Qgut/BW)
})
blockDisplay(*(equations[e] for e in [Qsto, Qsto1, Qsto2, Qgut, Ra]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Equation (8) from [Dalla Man et al. 2006]

In [10]:
equations.update({
    kempt  : sympy.Eq(kempt, kmin + (kmax-kmin)*(tanh(5*(Qsto-b*D)/(2*D*(1-b))) - tanh(5*(Qsto-c*D)/(2*D*c))+2)/2)
})
blockDisplay(*(equations[e] for e in [kempt]))

<IPython.core.display.Latex object>

##### Glucose utilization

Caution: constant K_mx is not given any value in the paper... (if I understand correctly, parameter estimation infers
a value of 0).

Note also that in the book "Modelling Methodology for Physiology and Medicine", another (more complex...) equation is given for Uid that doesn't use Kmx. 

Equations (14) (15) (18) and (19)

In [11]:
equations.update({
    Uii : sympy.Eq(Uii, Fcns),
    Uid : sympy.Eq(Uid, (Vm0*Gt + Vmx*X*Gt)/(Km0+Kmx*X+Gt)),
    X   : sympy.Eq(X.diff(), -p2U*X + p2U*(I-Ib)),
    U   : sympy.Eq(U, Uii+Uid)
})
blockDisplay(*(equations[e] for e in [Uii,Uid,X,U]))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

##### Insulin secretion

Equations (23) (24) (25) and (26).
_As in the matlab implementation we add a condition in the first branch of the equation for Spo: we use Gb instead of h since h=Gb in the matlab implementation._

In [42]:
# FIXME: remettre l'équation Spo correcte
equations.update({ 
    S   : sympy.Eq(S,γ*Ipo),
    Ipo : sympy.Eq(Ipo.diff(), -γ*Ipo + Spo),
    Y   : sympy.Eq(Y.diff(), 
                   sympy.Piecewise((-α*(Y-β*(G-h)), β*(G-h)>= -Sb),
                                   (-α*Y-α*Sb, True))),
    Spo : sympy.Eq(Spo, 
                   #sympy.Piecewise((Y+K*G.diff()+Sb, (G.diff()>0) & (G > h)),
                   sympy.Piecewise((Y+K*G.diff()+Sb, (G.diff()>0) & (G > 2)),
                                   (Y+Sb, True)))
})
blockDisplay(*(equations[e] for e in [S,Ipo,Y,Spo]))

RecursionError: maximum recursion depth exceeded while calling a Python object

##### Glucose Renal Excretion

Equation (27)

In [ ]:
equations.update({
    E : sympy.Eq(E, 
                sympy.Piecewise((ke1*(Gp-ke2), Gp>ke2),
                                (0, True)))
})
blockDisplay(equations[E])

### Insulin subsystem

Equations (3) and (11)

In [ ]:
equations.update({
    Il : sympy.Eq(Il.diff(), -(m1 + m3)*Il + m2*Ip + S),
    Ip : sympy.Eq(Ip.diff(), -(m2 + m4)*Ip + m1*Il),
    I  : sympy.Eq(I,Ip/VI),
    I1 : sympy.Eq(I1.diff(), -ki*(I1-I)),
    Id : sympy.Eq(Id.diff(), -ki*(Id-I1))
})
blockDisplay(*(equations[e] for e in [Il,Ip,I,I1,Id]))

Equations (4) and (5)

In [ ]:
equations.update({
    HE : sympy.Eq(HE, -m5*S + m6),
    m3 : sympy.Eq(m3, HE*m1/(1-HE))
})
blockDisplay(*(equations[e] for e in [HE, m3]))

## Parameters values

- The values for the parameters b and c used in the equation for kempt are difficult to find: in the [Dalla Man et al. 2007] paper (TABLE 1) there are 4 parameters (a,b,c,d) where I chose b and d to match, resp. b and c here (indeed, IMO these should be dimensionless).
- I assume that the value of Kmx is 0 (cite: "Note that, when fitting on our data [...] Kmx collapses to zero so that Km is no more dependent from X")
- according to  [Dalla Man et al. 2007] (p 1745) : "h has been set to the basal glucose concentration Gb to guarantee system steady state in basal condition"

In [ ]:
parameter_values = dict()
model.parameter_values = parameter_values

In [ ]:
parameter_values['normal'] = dict()
parameter_values['normal'].update({
    # glucose kinetics
    VG : 1.88,
    k1 : 0.065,
    k2 : 0.079,
    # insulin kinetics
    VI : 0.05,
    m1 : 0.19,
    m2 : 0.484,
    m4 : 0.194,
    m5 : 0.0304,
    m6 : 0.6471,
    HEb: 0.6,
    # rate of appearance
    kmax: 0.0558,
    kmin: 0.008,    
    kabs: 0.057,
    kgri: 0.0558,
    f: 0.9,
    b: 0.82,
    c: 0.01,
    # Endogenous production
    kp1: 2.7,
    kp2: 0.0021,
    kp3: 0.009,
    kp4: 0.0618,
    ki: 0.0079,
    # Utilization
    Fcns: 1, 
    Vm0: 2.5, 
    Vmx: 0.047,
    Km0: 225.59,
    Kmx: 0,
    p2U: 0.0331,
    # Secretion
    K: 2.3,
    α: 0.05,
    β: 0.11,
    γ: 0.5,
    # Renal Excretion
    ke1: 0.0005,
    ke2: 339,
    # Body weight (not given in Table 2 but in text, +- 1kg)
    BW: 78,
    # Basal glucose concentration (according to Fig. 5)
    Gb: 90
})

blockDisplay(parameter_values['normal'])

In [ ]:
parameter_values['diabetic'] = dict()
parameter_values['diabetic'].update({
    # glucose kinetics
    VG : 1.49,
    k1 : 0.042,
    k2 : 0.071,
    # insulin kinetics
    VI : 0.04,
    m1 : 0.379,
    m2 : 0.673,
    m4 : 0.269,
    m5 : 0.0526,
    m6 : 0.8118,
    HEb: 0.6,
    # rate of appearance
    kmax: 0.0465,
    kmin: 0.0076,    
    kabs: 0.023,
    kgri: 0.0465,
    f: 0.9,
    b: 0.68,
    c: 0.09,
    # Endogenous production
    kp1: 3.09,
    kp2: 0.0007,
    kp3: 0.005,
    kp4: 0.0786,
    ki: 0.0066,
    # Utilization
    Fcns: 1, 
    Vm0: 4.65, 
    Vmx: 0.034,
    Km0: 466.21,
    Kmx: 0,
    p2U: 0.0840,
    # Secretion
    K: 0.99,
    α: 0.013,
    β: 0.05,
    γ: 0.5,
    # Renal Excretion
    ke1: 0.0007,
    ke2: 269,
    # Body weight (not given in Table 2 but in text, +- 5kg)
    BW: 91,
    # Basal glucose concentration (according to Fig. 5)
    Gb: 160
})

display(parameter_values['diabetic'])

In [ ]:
parameter_values['RYGB'] = parameter_values['diabetic'].copy()
parameter_values['RYGB'].update({f : 0.5})
display(parameter_values['RYGB'])

## Initial conditions

Values of the variables at t=0. Note that only the variables defined by a differential equation need to be given  initial conditions explicitly. The initial conditions of the other variables are implicitly obtained by their equation. 

_As in the matlab implementation we fix D to 90000_

In [ ]:
model.initial_conditions = {
    Gp   : Gpb,  # eq (1)
    Gt   : Gtb,  # eq (1)
    G    : Gb,   # eq (1)
    EGP  : EGPb, # eq (10)
    Il   : Ilb,  # eq (3)
    Ip   : Ipb,  # eq (3)
    I    : Ib,   # eq (3)
    I1   : Ib,   # eq (11)
    Id   : Ib,   # eq (11)
    HE   : HEb,  # eq (4)
    Qsto : 0,    # eq (13)
    Qsto1: 0,    # eq (13)
    Qsto2: 0,    # eq (13)
    Qgut : 0,    # eq (13)
    Ra   : 0,    # eq (13)
    X    : 0,    # eq (18)
    Y    : 0,    # eq (26)
    Ipo  : Ipob,  # eq (24)
    D    : 90000
}

The initial conditions depending on basal values, in order to solve the ODEs we first need to analyze to system at steady state. 

# Steady state analysis

#### Essai de résolution des valeurs à steady state à l'aide des équations 2, 6-9, 12 et 20-22 du papier

In [ ]:
# model.steady_state = EmptyObject()
# model.steady_state.equations = dict()
# model.steady_state.equations.update({
#     EGPb: sympy.Eq(EGPb, Ub+Eb),
#     #m2: sympy.Eq(m2, (Sb/Ipb - m4/(1-HEb))*((1-HEb)/HEb)), 
#     m2: sympy.Eq(m3b, 3*Sb / (5*Ilb)), # We correct the equations (9) 
#     m4: sympy.Eq(m4,((2*Sb)/(5*Ipb))), # We correct the equations (9)
#     m6: sympy.Eq(m6,m5*Sb+HEb),
#     m3b: sympy.Eq(m3b,(HEb*m1)/(1-HEb)),
#     Sb: sympy.Eq(Sb,m3b*Ilb+m4*Ipb),
#     kp1: sympy.Eq(kp1, EGPb+kp2*Gpb+kp3*Ib+kp4*Ipob),
#     Gtb: sympy.Eq(Gtb, (Fcns-EGPb+k1*Gpb)/k2),
#     Ub: sympy.Eq(Ub, EGPb),
#     Fcns: sympy.Eq(Ub, Fcns+(Vm0*Gtb)/(Km0+Gtb)),
#     Vm0: sympy.Eq(Vm0, (EGPb-Fcns)*(Km0+Gtb)/Gtb)
# })
# model.steady_state.basal_variables = [EGPb,m2,m4,m6,m3b,Sb,kp1,Gtb,Ub,Fcns,Vm0]
# #model.steady_state.basal_variables = [EGPb,m4,m6,m3b,Sb,kp1,Gtb,Ub,Fcns,Vm0]
# blockDisplay(*(model.steady_state.equations[e] for e in model.steady_state.basal_variables))
# #blockDisplay(list(model.steady_state.equations.values()))

In [ ]:
# sympy.solve(list(model.steady_state.equations.values()),{Eb,EGPb,Sb,m3b,Ilb,Ipb,Ib,Gpb,Gtb,Ub,Ipob})

**These equations seems to be not enough to get a solution for Ub and Ipob (the value for HEb is given in the paper as 0.6).**

# Own steady state analysis

Bellow is an attempt to make our own steady state analysis. We succeed but obtain very complex (symbolic) basal values. One reason is probably that we don't consider this assumption that "the liver is responsible for 60% of insulin clearance in the steady state" that led, in the paper, to the equation (9).

In order to determine the initial conditions we need to infer the basal fluxes.
We just fix the amount ingested of glucose as 0. The other input variables (body weight) will subsequently be deleted. In the paper [Dalla Man et al. 2019], the authors give equations to infer the steady state values. Here, we first perform our own steady state analysis to infer these values. Then, we solve the equations given in the paper and finally compare the resulted values.  

In [ ]:
my_steady_state = EmptyObject()
conditions = EmptyObject()
model.my_steady_state = my_steady_state
my_steady_state.conditions = conditions

conditions.nostimuli = {
    D: 0
}

blockDisplay(
    'Conditions representing absence of external stimuli:',
)
model.my_steady_state.conditions.nostimuli

Let us set all the derivative to 0 (i.e. steady state).

In [ ]:
conditions.nochange = dict()

for ev in model.symbols.variables:
    conditions.nochange.update({ev.diff(): 0})

blockDisplay(
    'Conditions representing constantness of species concentration:',
)
model.my_steady_state.conditions.nochange


Substitute the derivatives with 0 and the function symbols (i.e. the variables) for their basal steady state constant symbol:

In [ ]:
variables_to_basal = { 
    x: y for (x,y) in list(zip(symbols.variables, symbols.variables_basal)) 
}
ss_equations = model.equations.copy()
ss_equations = \
    subs(ss_equations, my_steady_state.conditions.nochange)
ss_equations = \
    subs(ss_equations, my_steady_state.conditions.nostimuli)
ss_equations = \
    subs(ss_equations, variables_to_basal)
ss_equations.update({ h : sympy.Eq(h, Gb) })
blockDisplay(*list(ss_equations.values()))

We first break down the piecewise definitions for E and Y: 
- at basal steady state we can safely assume that glucose renal excretion E is null;
- replacing Gb by h in the definition of Y implies that only the first branch of the definition is always satisfied (sympy simplifies it automatically!)  

In [ ]:
model.my_steady_state.symbolic_basal_values = dict()
#model.my_steady_state.symbolic_basal_values.update({Gb:h, Eb:0})
model.my_steady_state.symbolic_basal_values.update({Eb:0})
ss_equations.update({E : sympy.Eq(Eb, 0)})
ss_equations = subs(ss_equations, {Eb:0,h:Gb})
blockDisplay(*list(ss_equations.values()))

#### Some usefull fonction to automate the resolution of the steady state equations:

In [ ]:
def solve_partial_equations(eqns, variables):
    tmp_equations = [
        eqns[e]
        for e in variables 
        if e in eqns.keys()
    ]
    return sympy.solve(tmp_equations,{ variables_to_basal[e] for e in variables })

def partial_update_of_steady_state(symbolic_basal_values, sol):
    for k in list(symbolic_basal_values.keys()):
        if type(symbolic_basal_values[k])!=int:
            symbolic_basal_values.update(
                {k:subs(symbolic_basal_values[k],sol)}
            )
    symbolic_basal_values.update(sol)
    
def partial_update_of_ss_equations(eqns,symbolic_basal_values):
    return subs(eqns,symbolic_basal_values)

#### We first solve the trivial equations for G, HE, I, kempt, Qgut, Qsto, Qsto1, Qsto2, Ra, S, Uii, X, Y, m3 and EGP

In [ ]:
tmp_solutions = solve_partial_equations(
                    ss_equations,
                    [Gp,HE,I,kempt,Qgut,Qsto,Qsto1,Qsto2,Ra,S,Uii,X,Y,m3,EGP]
                )
partial_update_of_steady_state(model.my_steady_state.symbolic_basal_values,tmp_solutions[0])
ss_equations = partial_update_of_ss_equations(ss_equations,model.my_steady_state.symbolic_basal_values)

In [ ]:
print("Partial steady state:")
blockDisplay(model.my_steady_state.symbolic_basal_values)

#### Solve sub-system for I1b, Idb and Ipb

In [ ]:
tmp_solutions = solve_partial_equations(
                    ss_equations,
                    [I1,Id,Ip]
                )
partial_update_of_steady_state(model.my_steady_state.symbolic_basal_values,tmp_solutions)
ss_equations = partial_update_of_ss_equations(ss_equations,model.my_steady_state.symbolic_basal_values)
display(model.my_steady_state.symbolic_basal_values)

#### Solve sub-system for Il, Ipo and Spo

In [ ]:
tmp_solutions = solve_partial_equations(
                    ss_equations,
                    [Il,Ipo,Spo]
                )
partial_update_of_steady_state(model.my_steady_state.symbolic_basal_values,tmp_solutions[0])
ss_equations = partial_update_of_ss_equations(ss_equations,model.my_steady_state.symbolic_basal_values)
display(model.my_steady_state.symbolic_basal_values)

#### Solve the sub-system for Ub and Uidb 

In [ ]:
tmp_solutions = solve_partial_equations(
                    ss_equations,
                    [U,Uid]
                )
partial_update_of_steady_state(model.my_steady_state.symbolic_basal_values,tmp_solutions)
ss_equations = partial_update_of_ss_equations(ss_equations,model.my_steady_state.symbolic_basal_values)
display(model.my_steady_state.symbolic_basal_values)

#### Solve the variable Spob

In [ ]:
tmp_solutions = sympy.solve(ss_equations[G],{Spob})

We obtain two symbolc solutions for Spob. For now, we keep both of them to solve Gt.  

In [ ]:
symbolic_basal_values0 = model.my_steady_state.symbolic_basal_values.copy()
ss_equations0 = ss_equations.copy()
partial_update_of_steady_state(symbolic_basal_values0,{Spob:tmp_solutions[0]})
ss_equations0 = partial_update_of_ss_equations(ss_equations0,symbolic_basal_values0)

In [ ]:
symbolic_basal_values1 = model.my_steady_state.symbolic_basal_values.copy()
ss_equations1 = ss_equations.copy()
partial_update_of_steady_state(symbolic_basal_values1,{Spob:tmp_solutions[1]})
ss_equations1 = partial_update_of_ss_equations(ss_equations1,symbolic_basal_values1)

#### Last step: solving the variable Gtb

In [ ]:
tmp_solutions = sympy.solve(ss_equations0[Gt],{Gtb})
tmp_solutions

We have two solutions of which only the second is valid (i.e. positive):

In [ ]:
print("first normal solution: " + str(tmp_solutions[0].subs(model.parameter_values['normal'])))
print("first diabetic solution: " + str(tmp_solutions[0].subs(model.parameter_values['diabetic'])))
print("first RYGB solution: " + str(tmp_solutions[0].subs(model.parameter_values['RYGB'])))
print("second normal solution: " + str(tmp_solutions[1].subs(model.parameter_values['normal'])))
print("second diabetic solution: " + str(tmp_solutions[1].subs(model.parameter_values['diabetic'])))
print("second RYGB solution: " + str(tmp_solutions[1].subs(model.parameter_values['RYGB'])))

We thus update both previous steady states with this solution:

In [ ]:
partial_update_of_steady_state(symbolic_basal_values0,{Gtb:tmp_solutions[1]})
partial_update_of_steady_state(symbolic_basal_values1,{Gtb:tmp_solutions[1]})

In [ ]:
print("first normal solution: " + str(symbolic_basal_values0[Spob].subs(model.parameter_values['normal'])))
print("first diabetic solution: " + str(symbolic_basal_values0[Spob].subs(model.parameter_values['diabetic'])))
print("first RYGB solution: " + str(symbolic_basal_values0[Spob].subs(model.parameter_values['RYGB'])))
print("second normal solution: " + str(symbolic_basal_values1[Spob].subs(model.parameter_values['normal'])))
print("second diabetic solution: " + str(symbolic_basal_values1[Spob].subs(model.parameter_values['diabetic'])))
print("second RYGB solution: " + str(symbolic_basal_values1[Spob].subs(model.parameter_values['RYGB'])))

Thus, the solution is symbolic_basal_values0:

In [ ]:
model.my_steady_state.symbolic_basal_values = symbolic_basal_values0

### Verification

Evaluating HEb for the normal values of the parameters given in the paper, we obtain a value close to 0.6 given in Table 1 of the paper! (h is fixed to Gb that is about 100/VG for normal subject and 160/VG for diabetic subject as depicted in top left curves of Fig 5)

In [ ]:
print("HEb for normal parameter values: " + 
    str(sympy.N(model.my_steady_state.symbolic_basal_values[HEb].subs(model.parameter_values['normal']).subs(Gpb,90))))
print("HEb for diabetic parameter values: " + 
    str(sympy.N(model.my_steady_state.symbolic_basal_values[HEb].subs(model.parameter_values['diabetic']).subs(Gpb,160))))
print("HEb for RYGB parameter values: " + 
    str(sympy.N(model.my_steady_state.symbolic_basal_values[HEb].subs(model.parameter_values['RYGB']).subs(Gpb,160))))

### Numerical basal (normal and diabetic) values

We fix the numerical values of the variables at steady state for both normal and diabetic subjects: 

In [ ]:
model.my_steady_state.numeric_basal_values = dict()
for subject in ['normal', 'diabetic', 'RYGB']:
    model.my_steady_state.numeric_basal_values[subject] = dict()
    for k in list(model.my_steady_state.symbolic_basal_values.keys()):
        if type(model.my_steady_state.symbolic_basal_values[k]) != int:
            model.my_steady_state.\
            numeric_basal_values[subject].\
            update({k:sympy.N(subs(model.my_steady_state.symbolic_basal_values[k],model.parameter_values[subject]))})

In [ ]:
model.my_steady_state.numeric_basal_values['normal']

In [ ]:
model.my_steady_state.numeric_basal_values['diabetic']

In [ ]:
model.my_steady_state.numeric_basal_values['RYGB']

# Numerical integration

Set of variables that will be integrated:

In [ ]:
numerical_simulation = EmptyObject()
numerical_simulation.variables = set(model.symbols.variables)
numerical_simulation.variables

Equations where all symbolic parameters are replaced by a numerical value. We first use the values for normal subjects. 

In [ ]:
values = {
    subject : model.parameter_values[subject].copy()
    for subject in ['normal', 'diabetic', 'RYGB']
}

for  subject in ['normal', 'diabetic', 'RYGB']:
    values[subject].update({h: model.parameter_values[subject][Gb]})

# FIXME: régler le problème de récursion!!!
# essayer de remplacer le "& G > h" par "& true" dans la définition de Spo pour voir si c'est un problème
# de substitution dans les expressions logiques

numerical_simulation.numeric_equations = dict()
for subject in ['normal', 'diabetic', 'RYGB']: 
    numerical_simulation.numeric_equations[subject] = model.equations.copy()
    numerical_simulation.numeric_equations[subject] = subs(numerical_simulation.numeric_equations[subject],
                                                           values[subject], maxRecursionDepth=20)
    numerical_simulation.numeric_equations[subject] = subs(numerical_simulation.numeric_equations[subject],
                                                           model.my_steady_state.numeric_basal_values[subject])
    numerical_simulation.numeric_equations[subject].update({D : sympy.Eq(D.diff(),0)})
numerical_simulation.variables.add(D)

We remove algebraic equations and keep only differential equations. We use a fonction that substitutes variables defined by an algebraic equation by the RHS of its definition.

In [ ]:
def remove_alg_eq(var, eqs):
    eqs.update({ k : eqs[k].subs(var,eqs[var].rhs) for k in eqs.keys() })
    del eqs[var]

In [ ]:
only_diff_equations = dict()
only_diff_equations['normal'] = numerical_simulation.numeric_equations['normal'].copy()
only_diff_equations['diabetic'] = numerical_simulation.numeric_equations['diabetic'].copy()
only_diff_equations['RYGB'] = numerical_simulation.numeric_equations['RYGB'].copy()
for subject in ['normal', 'diabetic', 'RYGB']:
    for v in [E,EGP,G,HE,I,kempt,Qsto,Ra,S,Spo,U,Uid,Uii,m3]:
         remove_alg_eq(v,only_diff_equations[subject])

A derivative (d/dt Gp) remains in the equation of Ipo. This is problematic for the numerical integration, so we substitute it by the rhs of the equation for Gp. We have to do it by hand, that is we walk through the syntactic expression and reconstruct it with the substituted term.

In [ ]:
term = dict()
for subject in ['normal','diabetic','RYGB']:
    term[subject] = only_diff_equations[subject][Ipo].rhs.doit()
    display(term[subject])

In [ ]:
for subject in ['normal','diabetic','RYGB']:
    only_diff_equations[subject].update({Ipo : 
        only_diff_equations[subject][Ipo].doit().subs(Gp.diff(),only_diff_equations[subject][Gp].rhs)
    })

In [ ]:
dGp = dict()
first_value = dict()
second_value = dict()
new_term = dict()
for subject in ['normal','diabetic','RYGB']:
    dGp[subject] = only_diff_equations[subject][Gp].rhs.simplify()
    first_value[subject] = term[subject].args[1].args[0][0].args[1].args[0]
    second_value[subject] = term[subject].args[1].args[0][0].args[0]
    new_term[subject] = (Y + first_value[subject] * dGp[subject] + second_value[subject]).simplify()
    display(new_term[subject])

In [ ]:
new_new_term = dict()
for subject in ['normal','diabetic','RYGB']:
    new_new_term[subject] = term[subject].args[1].func(term[subject].args[1].args[0].func(new_term[subject],dGp[subject]>0),
                                                   term[subject].args[1].args[1]).simplify()
    blockDisplay(new_new_term[subject])

In [ ]:
for subject in ['normal','diabetic','RYGB']:
    only_diff_equations[subject].update({Ipo: sympy.Eq(Ipo,term[subject].func(term[subject].args[0],new_new_term[subject]))})

Let's simplify the whole ODE:

In [ ]:
# for subject in ['normal','diabetic','RYGB']:
#     for k in list(only_diff_equations[subject].keys()):
#         only_diff_equations[subject].update({k:only_diff_equations[subject][k].simplify()})
#only_diff_equations

In [ ]:
eq_vars = tuple(only_diff_equations['normal'].keys())
display(eq_vars)

In [ ]:
only_diff_equations['normal'][Gp]

In [ ]:
for subject in ['normal','diabetic','RYGB']:
    only_diff_equations[subject].update({Qsto1 : sympy.Eq(Qsto1.diff(),only_diff_equations[subject][Qsto1].lhs)})
    only_diff_equations[subject][Qsto1]

In [ ]:
eqnf = { subject : tuple(
                       sympy.lambdify(
                         (t,) + eq_vars,
                          only_diff_equations[subject][ev].args[1].\
                           subs(DiracDelta(t),0.5 - sympy.tanh(10*(t-.3))/2),
                           modules='sympy'
                        ) for ev in eq_vars
                   ) for subject in ['normal', 'diabetic', 'RYGB']
       }

In [ ]:
eqnf['normal']

In [ ]:
eq_vars

In [ ]:
def numsys_normal(t, y, *kargs, **kwargs): return [eqn(t, *y) for eqn in eqnf['normal']]
def numsys_diabetic(t, y, *kargs, **kwargs): return [eqn(t, *y) for eqn in eqnf['diabetic']]
def numsys_RYGB(t, y, *kargs, **kwargs): return [eqn(t, *y) for eqn in eqnf['RYGB']]

In [ ]:
initial_conditions = dict()
for subject in ['normal', 'diabetic', 'RYGB']:
    initial_conditions[subject] = \
        tuple(
               (
                 model.initial_conditions[ev] 
                 if (type(model.initial_conditions[ev]) == int)
                 else model.my_steady_state.numeric_basal_values[subject][model.initial_conditions[ev]]
               ) for ev in eq_vars
            )
#initial_conditions

In [ ]:
display(only_diff_equations['diabetic'][Gp])
display(only_diff_equations['RYGB'][Gp])

## Simulations

In [ ]:
interv = [0, 600]
xp = scipy.linspace(*interv, 100)
sol_normal = solve_ivp(numsys_normal, interv, initial_conditions['normal'], method="LSODA")
sol_diabetic = solve_ivp(numsys_diabetic, interv, initial_conditions['diabetic'], method="LSODA")
sol_rygb = solve_ivp(numsys_RYGB, interv, initial_conditions['RYGB'], method="LSODA")

#### Plasma Glucose G(t)

* normal = blue
* diabetic = orange
* RYGB = green

In [ ]:
plot(sol_normal.t, sol_normal.y[0]/model.parameter_values['normal'][VG])
plot(sol_diabetic.t, sol_diabetic.y[0]/model.parameter_values['diabetic'][VG])
plot(sol_rygb.t, sol_rygb.y[0]/model.parameter_values['RYGB'][VG])
show()

#### Glucose rate of appearance Ra(t)

In [ ]:
plot(sol_normal.t, model.parameter_values['normal'][f]*model.parameter_values['normal'][kabs]*sol_normal.y[eq_vars.index(Qgut)]/model.parameter_values['normal'][BW])
plot(sol_diabetic.t, model.parameter_values['diabetic'][f]*model.parameter_values['diabetic'][kabs]*sol_diabetic.y[eq_vars.index(Qgut)]/model.parameter_values['diabetic'][BW])
plot(sol_rygb.t, model.parameter_values['RYGB'][f]*model.parameter_values['RYGB'][kabs]*sol_rygb.y[eq_vars.index(Qgut)]/model.parameter_values['RYGB'][BW])
show()

#### Plasma insulin I(t)

In [ ]:
plot(sol_normal.t, sol_normal.y[eq_vars.index(Ip)]/model.parameter_values['normal'][VI])
plot(sol_diabetic.t, sol_diabetic.y[eq_vars.index(Ip)]/model.parameter_values['diabetic'][VI])
plot(sol_rygb.t, sol_rygb.y[eq_vars.index(Ip)]/model.parameter_values['RYGB'][VI])
show()

#### Endogenous glucose production EGP(t)

In [ ]:
plot(sol_normal.t, model.parameter_values['normal'][kp1]-model.parameter_values['normal'][kp2]*sol_normal.y[eq_vars.index(Gp)]
             -model.parameter_values['normal'][kp3]*sol_normal.y[eq_vars.index(Id)]
             -model.parameter_values['normal'][kp4]*sol_normal.y[eq_vars.index(Ipo)])
plot(sol_diabetic.t, model.parameter_values['diabetic'][kp1]-model.parameter_values['diabetic'][kp2]*sol_diabetic.y[eq_vars.index(Gp)]
             -model.parameter_values['diabetic'][kp3]*sol_diabetic.y[eq_vars.index(Id)]
             -model.parameter_values['diabetic'][kp4]*sol_diabetic.y[eq_vars.index(Ipo)])
plot(sol_rygb.t, model.parameter_values['RYGB'][kp1]-model.parameter_values['RYGB'][kp2]*sol_rygb.y[eq_vars.index(Gp)]
             -model.parameter_values['RYGB'][kp3]*sol_rygb.y[eq_vars.index(Id)]
             -model.parameter_values['RYGB'][kp4]*sol_rygb.y[eq_vars.index(Ipo)])
show()

#### Insulin secretion S(t)

In [ ]:
plot(sol_normal.t, sol_normal.y[eq_vars.index(Ipo)]*model.parameter_values['normal'][γ])
plot(sol_diabetic.t, sol_diabetic.y[eq_vars.index(Ipo)]*model.parameter_values['diabetic'][γ])
plot(sol_rygb.t, sol_rygb.y[eq_vars.index(Ipo)]*model.parameter_values['RYGB'][γ])
show()

# Comparison with clinical data (mixed meal test)

In [ ]:
import csv
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

In [ ]:
data = []
with open('../Pattou_data/repasTests.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        try:
            for row in reader:
                data_row = []
                if (row['type operation'] == '1,00') and (row['diabete'] == '0,00'):
                   data_row = [locale.atof(row['mmtt-glyc'+x+'_mg_dl']) for x in ['30', '60', '90', '120', '180']]
                   data_row.insert(0,locale.atof(row['num_obese']))
                data.append(data_row)
        except:
            print('OUPS!')

Remove empty data rows:

In [ ]:
data = [x for x in data if x != []]

In [ ]:
data

In [ ]:
def display_data_diabetic(num):
    plot(sol_diabetic.t, sol_diabetic.y[0]/model.parameter_values['diabetic'][VG])
    plot([30,60,90,120,180], data[num][1:], linestyle = 'none', marker = 'o')
    print('num_obese = '+str(data[num][0]))

In [ ]:
def display_data_normal(num):
    plot(sol_normal.t, sol_normal.y[0]/model.parameter_values['normal'][VG])
    plot([30,60,90,120,180], data[num][1:], linestyle = 'none', marker = 'o')
    print('num_obese = '+str(data[num][0]))

In [ ]:
display_data_normal(4)